# Retrieve data (crystal structures) from external sources
Time: 2 mins

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
if not is_dbenv_loaded():
    load_dbenv()
from aiida.tools.dbimporters import DbImporterFactory
from ase.visualize import view
import spglib
import nglview

## Import a structure from the ICSD database (https://www.fiz-karlsruhe.de/icsd.html)

<div class="alert alert-block alert-warning">
This test needs that you have access to a computer where you have installed the ICSD database, installed from the ICSD CD that you can buy.<br>
If you do not have it, this demo will not work.
</div>

#### Connect (with ssh tunnelling) to the server hosting the database

In [ ]:
# Change these to point to the correct username and machine on which you installed the ICSD server.
# You need also to setup passwordless SSH connection for this to work
remote_username = "aiidademo"
remote_server = "theossrv2.epfl.ch"

In [ ]:
!ssh -fN -L 3306:localhost:3306 -L 8010:localhost:80 {remote_username}@{remote_server} > /dev/null 2>&1

#### Load the ICSD importer class and instantiate it

In [ ]:
importer_class = DbImporterFactory('icsd')
importer_parameters = {'server': 'http://localhost:8010/',
                   'host': '127.0.0.1',
                   'db': 'icsd',
                   'passwd': 'sql'
                  }
importer = importer_class(**importer_parameters)
importer

#### Query the ICSD for a given database entry

In [ ]:
icsd_id = '617290' # graphite

In [ ]:
query_results = importer.query(id=icsd_id)

In [ ]:
print len(query_results)

#### Get the cif file corresponding to the entry, and convert it to an AiiDA structure

In [ ]:
entry = query_results.at(0)
cif = entry.get_cif_node()
print cif.get_formulae()

In [ ]:
structure = cif._get_aiida_structure(converter='pymatgen')
print structure.get_formula()
print structure.get_ase()

#### Get the spacegroup of the structure with spglib

In [ ]:
print spglib.get_spacegroup(structure.get_ase(),symprec=5e-3)

#### Store the structure and prints its pk

In [ ]:
structure.store()
structure.pk

#### Now we can visualize the structure

In [ ]:
view = nglview.show_ase(structure.get_ase()*[4,4,1])
view

## Other databases

You can also try <a href="retrieve_structure_from_external_db_COD.ipynb">a similar demo to import structures from the open-access COD (Crystallography Open Database)</a>.